In [13]:
import pandas as pd
import sklearn
print(sklearn.__version__)

1.6.1


In [10]:
#Librerías
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
# Cargar el archivo CSV
df = pd.read_csv("clientes_preprocesamiento.csv")

# Mostrar las primeras filas del DataFrame
print(df.head())

   ID  Edad     Genero  Ciudad  Ingresos_Mensuales  Frecuencia_Compra  \
0   1   NaN  Masculino  Cuenca              2946.0                  3   
1   2   NaN        NaN     NaN              1340.0                  8   
2   3   NaN   Femenino  Ambato              1052.0                  8   
3   4   NaN        NaN   Quito              2321.0                  2   
4   5  35.0        NaN   Quito              1522.0                  2   

   Ultima_Compra_dias  Compra  
0                 103       0  
1                  72       0  
2                 173       1  
3                   4       0  
4                 166       1  


In [4]:
# Realizar un backup para trabajar sobre este 
df_copia = df.copy()

In [6]:
#Recolección y exploración de datos
# Ver columnas y tipos de datos
print(df_copia.info())

# Ver primeros registros
print(df_copia.head())

# Estadísticas generales de columnas numéricas
print(df_copia.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  3000 non-null   int64  
 1   Edad                1001 non-null   float64
 2   Genero              1981 non-null   object 
 3   Ciudad              2368 non-null   object 
 4   Ingresos_Mensuales  2968 non-null   float64
 5   Frecuencia_Compra   3000 non-null   int64  
 6   Ultima_Compra_dias  3000 non-null   int64  
 7   Compra              3000 non-null   int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 187.6+ KB
None
   ID  Edad     Genero  Ciudad  Ingresos_Mensuales  Frecuencia_Compra  \
0   1   NaN  Masculino  Cuenca              2946.0                  3   
1   2   NaN        NaN     NaN              1340.0                  8   
2   3   NaN   Femenino  Ambato              1052.0                  8   
3   4   NaN        NaN   Quito             

In [8]:
# Contabiliar el número de nulos por columna para la posterior limpieza
df_copia.columns
df_copia.isnull().sum()

ID                       0
Edad                  1999
Genero                1019
Ciudad                 632
Ingresos_Mensuales      32
Frecuencia_Compra        0
Ultima_Compra_dias       0
Compra                   0
dtype: int64

In [9]:
# Reemplazar valores nulos en todo el DataFrame
df_copia.fillna("sin info", inplace=True)
df_copia

C:\Users\Windows\AppData\Local\Temp\ipykernel_15332\2109874915.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'sin info' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_copia.fillna("sin info", inplace=True)


,ID,Edad,Genero,Ciudad,Ingresos_Mensuales,Frecuencia_Compra,Ultima_Compra_dias,Compra
0,1,sin info,Masculino,Cuenca,2946.0,3,103,0
1,2,sin info,sin info,sin info,1340.0,8,72,0
2,3,sin info,Femenino,Ambato,1052.0,8,173,1
3,4,sin info,sin info,Quito,2321.0,2,4,0
4,5,35.0,sin info,Quito,1522.0,2,166,1
...,...,...,...,...,...,...,...,...
2995,2996,42.0,sin info,Guayaquil,1898.0,2,168,1
2996,2997,sin info,sin info,Ambato,1570.0,7,21,1
2997,2998,sin info,sin info,Quito,3812.0,9,146,0
2998,2999,sin info,Masculino,Cuenca,3083.0,15,112,1


In [21]:
# Codificar la variable objetivo (Género)
encoder_genero = LabelEncoder()
df_copia["Genero"] = encoder_genero.fit_transform(df["Genero"])  # Femenino = 0, Masculino = 1

# Codificar la variable de entrada (Ciudad) con OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
ciudad_encoded = encoder.fit_transform(df[["Ciudad"]])

# Convertir a DataFrame y unirlo con el original
ciudad_df = pd.DataFrame(ciudad_encoded, columns=encoder.get_feature_names_out(["Ciudad"]))
df_copia = pd.concat([df_copia, ciudad_df], axis=1)

# Definir X e Y
X = df[ciudad_df.columns]  # Todas las columnas generadas por OneHotEncoder
Y = df["Genero"]

# Dividir datos en entrenamiento y prueba (80% - 20%)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Inicializar modelo de clasificación
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, Y_train)

# Predicción
predicciones = modelo.predict(X_test)

# Evaluar precisión
precision = accuracy_score(Y_test, predicciones)
print(f"Precisión del modelo: {precision:.2f}")

KeyError: "None of [Index(['Ciudad_Ambato', 'Ciudad_Cuenca', 'Ciudad_Guayaquil', 'Ciudad_Quito',\n       'Ciudad_nan'],\n      dtype='object')] are in the [columns]"